In [ ]:
%matplotlib notebook
%precision 3

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [9.5, 7]
import datetime
import pandas



In [ ]:
prefix="/home/ahu/git/gazproject/"

In [ ]:
def cleanUp(v):
    v["timestamp"]=pandas.to_datetime(v["periodTo"])
    v["timestamp"]=v["timestamp"].dt.strftime("%Y-%m-%d 06:00:00")
    v["timestamp"]=pandas.to_datetime(v["timestamp"])
    v.sort_values(["timestamp"], inplace=True)
    v.set_index("timestamp", inplace=True)
    
uzhgorod = pandas.read_json(prefix+"uzhgorod-data.json")
nordstreamF = pandas.read_json(prefix+"nordstream-fluxsys-data.json")
nordstreamO = pandas.read_json(prefix+"nordstream-opal-data.json")
yamalK = pandas.read_json(prefix+"yamal-kondratki-data.json")
yamalW = pandas.read_json(prefix+"yamal-wysokoje-data.json")
hermanowice = pandas.read_json(prefix+"hermanowice-data.json")

cleanUp(uzhgorod)
cleanUp(nordstreamF)
cleanUp(nordstreamO)
cleanUp(yamalK)
cleanUp(yamalW)
cleanUp(hermanowice)
begindate=max([uzhgorod.index.min(), nordstreamF.index.min(), nordstreamO.index.min(), yamalW.index.min(), yamalK.index.min(), hermanowice.index.min()])

enddate=min([uzhgorod.index.max(), nordstreamF.index.max(), nordstreamO.index.max(), yamalW.index.max(), yamalK.index.max(), hermanowice.index.max()])
enddate

In [ ]:
plt.figure()
plt.plot(uzhgorod.value/24/1000000, label="Uzhgorod")
plt.plot(nordstreamF.value/24/1000000, label="Nordstream Greifswald Fluxsys")
plt.plot(nordstreamO.value/24/1000000, label="Nordstream Greifswald Opal")
plt.plot(yamalK.value/24/1000000, label="Yamal Kondratki")
plt.plot(yamalW.value/24/1000000, label="Yamal Wysokoje")
plt.plot(hermanowice.value/24/1000000, label="Hermanowice")
plt.xlim(begindate, enddate)
plt.plot((uzhgorod.value+yamalK.value+yamalW.value+nordstreamF.value+nordstreamO.value+hermanowice.value)/24000000, label="Sum")

plt.ylabel("GW")
plt.legend()
plt.title("Russian natural gas flow to Europe")
plt.xticks(rotation=25)

plt.grid()

In [ ]:
def makeGraph(fname, limit=-1):
    global yamalK, yamalW, uzhgorod, nordstreamF, nodstreamO, hermanowice
    plt.figure()
    yamalK=yamalK[begindate:enddate]
    yamalW=yamalW[begindate:enddate]

    pal = ["#FFD700", "#0057B8", "#44344f", "#98a6d4", "#5b9279", "#c2f970"]
    labels=["Uzhgorod (Ukraine)", "Nordstream Fluxsys", "Nordstream OPAL", "Yamal Kondratki", "Yamal Wysokoje", "Hermanowice"]
    locbegindate = begindate
    if limit >= 0:
        locbegindate = enddate - datetime.timedelta(days=7)
    
    plt.xlim(locbegindate, enddate)
    plt.stackplot(uzhgorod.index,
              uzhgorod.value/24000000, nordstreamF.value/24000000, nordstreamO.value/24000000,
              yamalK.value/24000000,
              yamalW.value/24000000, hermanowice.value/24000000,
            colors=pal, labels=labels)
    plt.ylabel("Gigawatt")
    plt.title("Russian natural gas flow to Europe, last data point: "+enddate.strftime("%Y-%m-%d %H:%M"))
    plt.xticks(rotation=25)
    if(limit < 0):
        labels = labels + list(["Start of invasion"])
        plt.axvline(datetime.date(2022, 2, 24), ls=':', color='red', label="Start of invasion")
    plt.legend(reversed(plt.legend().legendHandles), reversed(labels), loc=2)

    plt.grid()
    plt.savefig(fname)
    
makeGraph(prefix+"/russian-gas.png")
makeGraph(prefix+"/russian-gas-week.png", 7)